# Conda 환경을 통한 간단한 PyTorch 인공 신경망 학습하기

### 데이터셋 생성 및 시각화

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from torch.utils.data import TensorDataset, DataLoader

# 2D classification dataset 생성
X, y = make_classification(n_samples=1000, n_features=2, n_informative=2, n_redundant=0, n_classes=2, random_state=42)

# Tensor로 변환
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# 데이터셋 시각화
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', edgecolor='k')
plt.title('Generated Dataset')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.show()

# Dataset & Dataloader 생성
dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


### PyTorch 모델 정의

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(2, 16)  # 입력: 2D 데이터
        self.fc2 = nn.Linear(16, 2)  # 출력: 2 클래스

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleNN()


### 모델 학습

In [ ]:
# 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 모델 학습
epochs = 20
for epoch in range(epochs):
    running_loss = 0.0
    for data in dataloader:
        inputs, labels = data

        # Optimizer 초기화
        optimizer.zero_grad()

        # Forward
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward 및 가중치 업데이트
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(dataloader):.4f}")


### 학습한 모델의 Classification Region 시각화

In [ ]:
import numpy as np

# 전체 2D 공간에 대해 예측
def plot_decision_boundary(model, X, y):
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                         np.arange(y_min, y_max, 0.01))

    grid = np.c_[xx.ravel(), yy.ravel()]
    grid_tensor = torch.tensor(grid, dtype=torch.float32)
    Z = model(grid_tensor)
    _, predicted = torch.max(Z, 1)
    Z = predicted.reshape(xx.shape)

    plt.contourf(xx, yy, Z, cmap="coolwarm", alpha=0.8)
    plt.scatter(X[:, 0], X[:, 1], c=y, edgecolor='k', cmap="coolwarm")
    plt.title('Decision Boundary')
    plt.show()

plot_decision_boundary(model, X, y)
